In [2]:
!pip install seaborn
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 8.6 MB/s eta 0:00:00


In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')

import sys, importlib
module_name = "Functions"
if module_name in sys.modules:
    del sys.modules[module_name]
Functions = importlib.import_module(module_name)

from Functions import *

import esm
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd 
import numpy as np
import torch

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO

from Bio import Entrez
from Bio import SeqIO

from transformers import EsmForMaskedLM, T5EncoderModel,T5Tokenizer,DataCollatorForLanguageModeling
from transformers.modeling_outputs import MaskedLMOutput


# Run code

In [7]:


model_raw, alphabet = esm.pretrained.load_model_and_alphabet('esm2_t36_3B_UR50D')
model_raw.eval()
batch_converter = alphabet.get_batch_converter()
modnam="/home3/oml4h/hugging_face_downloads/model_weights_topublish/ESM2-H3"
model = EsmForMaskedLM.from_pretrained(modnam)


device = torch.device("cpu")
if torch.cuda.is_available():
    model =  model.to(device)
    print("Transferred model to GPU")

Transferred model to GPU


In [8]:
# Entrez.email = "sample@example.org"

# handle = Entrez.efetch(db="nucleotide",
#                        id="NC_045512.2",
#                        rettype="gb",
#                        retmode="gb")
# whole_sequence = SeqIO.read(handle, "genbank")
model_layers = 36

In [9]:
sequences = read_sequences_to_dict('/home2/oml4h/PLM_SARS-CoV-2/Sequences/huH3N2_HA_CDS.translated.fas')
sequences

{'EPI2178977|HA|A/Thailand/8/2022|EPI_ISL_14991375|J': 'MKAIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQISGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNICIX',
 'EPI2096148|HA|A/Massachusetts/18/2022|EPI_ISL_13897082|J': 'MKAIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQN

In [10]:
ids=list(sequences.keys())




In [11]:
J_indexed_muts = [m for m in get_reference_mutations(ref = sequences[ids[0]],mut = sequences[ids[len(ids)-1]]) if "-" not in m  ] 
K_indexed_muts = [m for m in get_mutations(sequences[ids[0]],sequences[ids[len(ids)-1]]) if "del" not in m and '-' not in m  ] 
print("K->J mutations")
print(K_indexed_muts)

K->J mutations
['K18N', 'N138D', 'T151K', 'S160N', 'N174D', 'I176K', 'Q189R', 'K205R', 'K292E', 'T344A', 'S394N']


In [12]:
mutation_dictionary  = {k:J_indexed_muts[i] for i,k in enumerate(K_indexed_muts)}

In [28]:
# get info on model layers:
model_layers_info = model.config.to_dict()
print(model_layers_info)
#print how many layers the model has
print("Number of layers in the model: ", model_layers_info['num_hidden_layers'] + 1)  # +1 for the embedding layer

{'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'float32', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty': 0.0, 'temperature': 1.0, 'top_k': 50, 'top_p': 1.0, 'typical_p': 1.0, 'repetition_penalty': 1.0, 'length_penalty': 1.0, 'no_repeat_ngram_size': 0, 'encoder_no_repeat_ngram_size': 0, 'bad_words_ids': None, 'num_return_sequences': 1, 'output_scores': False, 'return_dict_in_generate': False, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'remove_invalid_values': False, 'exponential_decay_length_penalty': None, 'suppress_tokens': None, 'begin_supp

In [34]:
# 1. Redefine embed_sequence to use Hugging Face syntax
def embed_sequence(sequence, model, device, model_layers, batch_converter, alphabet):
    # Sequences to embed
    sequence_data = [('base', sequence)]
    
    # Get tokens
    batch_labels, batch_strs, batch_tokens = batch_converter(sequence_data)
    batch_len = (batch_tokens != alphabet.padding_idx).sum(1)[0]

    # Move tokens to GPU
    if torch.cuda.is_available():
        batch_tokens = batch_tokens.to(device=device, non_blocking=True)

    with torch.no_grad():
        # FIX: Use output_hidden_states instead of repr_layers
        results = model(batch_tokens, output_hidden_states=True)
    del batch_tokens

    # FIX: Handle Layer Indexing
    # HF hidden_states is a tuple: (embeddings, layer_1, ... layer_N)
    # Length is num_layers + 1.
    # If model_layers is too high (e.g. 33 for a 30-layer model), default to the last layer (-1).
    
    try:
        token_representation = results.hidden_states[model_layers][0]
    except IndexError:
        print(f"Warning: Requested layer {model_layers} is out of bounds for this model. Using last layer.")
        token_representation = results.hidden_states[-1][0]

    full_embedding = token_representation[1:batch_len - 1].cpu()
    base_mean_embedding = token_representation[1 : batch_len - 1].mean(0).cpu()

    # FIX: Access logits directly
    lsoftmax = torch.nn.LogSoftmax(dim=1)
    base_logits = lsoftmax((results.logits[0]).to(device="cpu"))
    
    return results, base_logits, base_mean_embedding, full_embedding

# 2. Redefine process_protein_sequence to call the NEW embed_sequence above
def process_protein_sequence(sequence, model, model_layers, batch_converter, alphabet, device):
    # Embed Sequence
    # This will now call the updated function defined in this cell, not the one in Functions.py
    results, base_logits, base_mean_embedding, full_embedding = embed_sequence(
        sequence, model, device, model_layers, batch_converter, alphabet
    )
    
    results_dict = {}
    results_dict["Mean_Embedding"] = base_mean_embedding.tolist()
    results_dict["Logits"] = base_logits.tolist()
    
    # Recalculate grammaticality if needed, or pass through
    # Assuming get_sequence_grammaticality is available in your scope
    try:
        results_dict["sequence_grammaticality"] = get_sequence_grammaticality(sequence, base_logits, alphabet)
    except NameError:
        print("Warning: get_sequence_grammaticality function not found in current scope.")
        results_dict["sequence_grammaticality"] = None
        
    return results_dict

def embed_protein_sequences(protein_sequences,reference_protein,coding_region_name,model,model_layers,device,batch_converter,alphabet,scores=False):
    #Embed Reference Protein Sequence
    results, reference_logits, reference_mean_embedding, full_embedding = embed_sequence(reference_protein,model,device,model_layers,batch_converter,alphabet)
    
    embeddings = {}
    
    word_pos_prob = {}
    for pos in range(len(reference_protein)):
        for word in alphabet.all_toks:
            word_idx = alphabet.get_idx(word)
            prob = reference_logits[pos + 1, word_idx]
            word_pos_prob[(word, pos)] = prob

    embeddings['Reference'] = {}
    embeddings['Reference'][coding_region_name] = {"Mean_Embedding":reference_mean_embedding.tolist(),
                                        "Logits":reference_logits.tolist(),
                                        "sequence_grammaticality":get_sequence_grammaticality(reference_protein,reference_logits,alphabet)
                                        
                                    }
    #Process Fasta Files       
    fasta_sequences = protein_sequences
    
    i = 0
    print(fasta_sequences)
    for fasta in fasta_sequences:
        print(fasta_sequences)
        name, sequence = fasta[0], str(fasta[1])
        print(name)
        

        #Scores work by creating sequences with no insertions or deletions, then calculating changes from this sequence
        #These embeddings are thus not "real" sequences 
        if scores == True:
            print(len(reference_protein),len(sequence))
            mutations = get_mutations(reference_protein,sequence)
            #get mutation position
            mut_pos= [int(mut[1:-1])-1 for mut in mutations if "del" not in mut ]
            mutation_only_sequence = mutate_sequence(reference_protein,mutations)
            embeddings[name] = {coding_region_name:process_protein_sequence(mutation_only_sequence,model,model_layers,batch_converter,alphabet,device)}
            
            # L1/Manhattan Distance between mean embeddings used for the semantic change
            semantic_change = float(sum(abs(target-base) for target, base in zip(reference_mean_embedding,embeddings[name][coding_region_name]["Mean_Embedding"])))
            gm, ev = grammaticality_and_evolutionary_index(word_pos_prob, reference_protein, mutations)
            print('Semantic score: ', semantic_change)
            print('Grammaticality: ', gm)
            print('Relative Grammaticality: ', ev)
            print('Probability: ', np.exp(gm))
            just_mutations = [mut for mut in mutations if "del" not in mut ]
            mutations_string = str([mut for mut in mutations if "del" not in mut ]).strip("[").strip("]").replace("'", "")
            deletions_string = str([mut for mut in mutations if "del" in mut ]).strip("[").strip("]").replace("'", "")
            
            embeddings[name][coding_region_name]["label"] = name
            embeddings[name][coding_region_name]["semantic_score"] = semantic_change
            embeddings[name][coding_region_name]["grammaticality"] = gm
            embeddings[name][coding_region_name]["relative_grammaticality"] = ev

            #Probability of whole sequence including mutation
            embeddings[name][coding_region_name]['sequence_grammaticality'] = get_sequence_grammaticality(sequence,embeddings[name][coding_region_name]['Logits'],alphabet)
            #narrow sense gramaticallity excludes the mutation- so looks at how whole sequence shifts excluding the focal site
            embeddings[name][coding_region_name]['narrow_sequence_grammaticality'] = get_sequence_grammaticality(sequence,embeddings[name][coding_region_name]['Logits'],alphabet,mask_pos=mut_pos)
            print('Sequence Grammaticality: ', embeddings[name][coding_region_name]['sequence_grammaticality'])
            #Probability ratio between the mutant sequence and the reference sequence
            embeddings[name][coding_region_name]['relative_sequence_grammaticality'] = embeddings[name][coding_region_name]['sequence_grammaticality']-embeddings['Reference'][coding_region_name]['sequence_grammaticality']
            #get probability of all the un-mutated sites
            ref_narrow=get_sequence_grammaticality(reference_protein,reference_logits,alphabet,mask_pos=mut_pos)
            embeddings[name][coding_region_name]['relative_narrow_sequence_grammaticality'] = embeddings[name][coding_region_name]['narrow_sequence_grammaticality']-ref_narrow

            embeddings[name][coding_region_name]["probability"] = np.exp(gm)

            embeddings[name][coding_region_name]["mutation_count"] = len(just_mutations)
            embeddings[name][coding_region_name]["mutations"] = mutations_string
            embeddings[name][coding_region_name]["deletions(not_included_in_scores)"] = deletions_string
            
        #If no need for scores, sequences can be embedded as is with insertions, deletions truncations etc
        else:
            embeddings[coding_region_name][name] = process_protein_sequence(str(fasta.seq),model,model_layers,batch_converter,alphabet,device)
        i+=1
    return embeddings


In [39]:
# backbone = "Wuhan-Hu-1"
backbone = ids[len(ids)-1]
all_mutations = {}
mut_info_rows=[]

#pd.DataFrame(columns=["Mutation","rel_grammar","rel_seq_grammar","semanatic_score","probability","Backbone"])

reference_spike_sequence = sequences[backbone]
#loop through each node in the tree except the last one (lineage of interest)
for backbone_i in range(len(ids)-1):
    backbone = ids[backbone_i]
    print("Calculating mutations on backbone:",backbone)
    reference_spike_sequence = sequences[backbone]
    #calculate mutation differences between node and K lineage
    J_indexed_muts = [m for m in get_reference_mutations(ref = sequences[backbone],mut = sequences[ids[len(ids)-1]]) if "-" not in m  ] 
    K_indexed_muts = [m for m in get_mutations(sequences[backbone],sequences[ids[len(ids)-1]]) if "del" not in m and '-' not in m  ] 
    mutation_dictionary  = {k:J_indexed_muts[i] for i,k in enumerate(K_indexed_muts)}
    print("K->J mutations")
    print(K_indexed_muts)
    #loop through each mutation and calculate the scores on the focal node
    for mut in mutation_dictionary.keys():
        if backbone ==  ids[len(ids)-1]:
            sequence = revert_sequence(reference_spike_sequence,[mut])
        else:
            sequence = mutate_sequence(reference_spike_sequence,[mut])
        reference = reference_spike_sequence
        mutations = embed_protein_sequences(
            [[mut,sequence.replace("-","")]],
            reference.replace("-",""),
            'S:0',
            model,
            model_layers,
            device,
            batch_converter,
            alphabet,
            scores=True)
   
        
        #append mutation info to dataframe
        new_row={"Mutation":mut,"rel_grammar":mutations[mut]["S:0"]["relative_grammaticality"],
                                    "rel_seq_grammar":mutations[mut]["S:0"]["relative_sequence_grammaticality"],
                                    "narrow_seq_grammar":mutations[mut]["S:0"]["narrow_sequence_grammaticality"],
                                    "relative_narrow_seq_grammar":mutations[mut]["S:0"]["relative_narrow_sequence_grammaticality"],
                                    "semanatic_score":mutations[mut]["S:0"]["semantic_score"],
                                    "probability":mutations[mut]["S:0"]["probability"],
                                    "Backbone":backbone}
        mut_info_rows.append(new_row)
        all_mutations[mut] = mutations
    
mut_info=pd.DataFrame(mut_info_rows)

Calculating mutations on backbone: EPI2178977|HA|A/Thailand/8/2022|EPI_ISL_14991375|J
K->J mutations
['K18N', 'N138D', 'T151K', 'S160N', 'N174D', 'I176K', 'Q189R', 'K205R', 'K292E', 'T344A', 'S394N']
[['K18N', 'MKAIIALSNILCLVFAQNIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQISGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNICIX']]
[['K18N', 'MKAIIALSNILCLVFAQNIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQ

In [41]:
#
mut_info["lineage_backbone"]=mut_info["Backbone"].str.split("|").str[-1]
mut_info
mut_info.to_csv("/home2/oml4h/PLM_SARS-CoV-2/Results/H3_epistasis_mutation_info_spyros_model.csv",index=False)
# print(mut_info.loc[mut_info['Mutation']=='I176K'])
# print(mut_info.loc[mut_info['Mutation']=='I151K'])

In [ ]:
megatable = []

for k in mutation_dictionary.keys():
    if k == 'Reference':
        continue
    relative_grammaticality = all_mutations[k][k]['S:0']["relative_grammaticality"]
    relative_sequence_grammaticality = all_mutations[k][k]['S:0']["relative_sequence_grammaticality"]
    semantic_score = all_mutations[k][k]['S:0']["semantic_score"]

    mutations = pd.DataFrame({'mutation':k,
                            'relative_grammaticality':relative_grammaticality,
                            'relative_sequence_grammaticality':relative_sequence_grammaticality,
                            'semantic_score':semantic_score,},index=[0])
    mutations.mutations = k
    megatable.append(mutations)
megatable_df = pd.concat(megatable)

if backbone == ids[len(ids)-1]:
    for mut in  mutation_dictionary.keys():
        megatable_df.mutation = megatable_df.mutation.str.replace(mut,mutation_dictionary[mut])
megatable_df

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler((0,1))

megatable_df['scaled_relative_grammaticality'] = mms.fit_transform(-megatable_df.relative_grammaticality.values.reshape(-1,1)).ravel()
megatable_df['scaled_relative_sequence_grammaticality'] = mms.fit_transform(-megatable_df.relative_sequence_grammaticality.values.reshape(-1,1)).ravel()

In [ ]:
megatable_df.sort_values('relative_grammaticality',ascending=False)

In [ ]:
import torch
lsoftmax = torch.nn.LogSoftmax(dim=1)

In [ ]:
def outside_std(column):
    filtered_vals = []
    for c in column:
        if c >=(column.mean() + (column.std()*2)) or c <= (column.mean() - (column.std()*2)):
            filtered_vals.append(c)
        else:
            filtered_vals.append(np.nan)
    return filtered_vals


In [ ]:
reference_logits_table = []
for k in all_mutations.keys():
    print(all_mutations[k].keys())
    backbone_logits = all_mutations[k]['Reference']['S:0']['Logits']
    backbone_logits = format_logits(backbone_logits,alphabet)
    backbone_logits = remap_logits(backbone_logits,sequences['Wuhan'], reference_spike_sequence)
    backbone_ref_seq_logits = [row[row.Sequence] if row.Sequence != '-' else np.nan for i,row in backbone_logits.iterrows()]

    for mutation in all_mutations[k].keys():
        if mutation != 'Reference':
            if backbone == 'BA.1':
                mutated_sequence = revert_sequence(reference_spike_sequence,[mutation])
            else:
                mutated_sequence = mutate_sequence(reference_spike_sequence,[mutation])

            mutated_logits = format_logits(all_mutations[k][mutation]['S:0']['Logits'],alphabet)
            mutated_logits = remap_logits(mutated_logits,sequences['Wuhan'], mutated_sequence, )
            mutated_seq_logits = [row[row.Sequence]  if row.Sequence != '-' else np.nan for i,row in mutated_logits.iterrows()]
            
            if backbone == 'BA.1':
                # Difference is backbone - mutation here since the "mutation" is actually a reversion
                difference = np.exp(backbone_ref_seq_logits) -  np.exp(mutated_seq_logits)
                pllr =  np.array(backbone_ref_seq_logits)- np.array(mutated_seq_logits)
            else:
                difference =  np.exp(mutated_seq_logits) - np.exp(backbone_ref_seq_logits)
                pllr =  np.array(mutated_seq_logits) - np.array(backbone_ref_seq_logits)

            df = pd.DataFrame(
                {'source_sequence':list(backbone_logits.Sequence),
                'target_sequence':list(mutated_logits.Sequence),
                'source_logits':backbone_ref_seq_logits,
                'target_logits':mutated_seq_logits,
                'change':difference,
                'ratio':pllr,
                'position':range(1,len(difference)+1)})
                
            df['masked_change'] =[row.change if row.source_sequence == row.target_sequence else np.nan for i, row in df.iterrows()]
            df['masked_ratio'] =[row.ratio if row.source_sequence == row.target_sequence else np.nan for i, row in df.iterrows()]
            
            local_max_change = df.masked_change.mean()+(df.masked_change.std()*2)
            local_min_change = df.masked_change.mean()-(df.masked_change.std()*2)

            local_max_ratio = df.masked_ratio.mean()+(df.masked_ratio.std()*2)
            local_min_ratio = df.masked_ratio.mean()-(df.masked_ratio.std()*2)

            df['significant_masked_change_local'] = [ check_valid(m,local_min_change,local_max_change) for m in df. masked_change]
            df['significant_masked_ratio_local'] = [ check_valid(m,local_min_ratio,local_max_ratio) for m in df.masked_ratio]

            df['mutation'] = mutation_dictionary[mutation]
            reference_logits_table.append(df)
reference_logits_table = pd.concat(reference_logits_table,axis=0)

max_change = reference_logits_table.masked_change.mean()+(reference_logits_table.masked_change.std()*2)
min_change = reference_logits_table.masked_change.mean()-(reference_logits_table.masked_change.std()*2)

max_ratio = reference_logits_table.masked_ratio.mean()+(reference_logits_table.masked_ratio.std()*2)
min_ratio = reference_logits_table.masked_ratio.mean()-(reference_logits_table.masked_ratio.std()*2)

reference_logits_table['significant_masked_change'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change]
reference_logits_table['significant_masked_ratio'] = [ check_valid(m,min_ratio,max_ratio) for m in reference_logits_table.masked_ratio]

max_change = reference_logits_table.masked_change.round(4).mean()+(reference_logits_table.masked_change.round(4).std()*2)
min_change = reference_logits_table.masked_change.round(4).mean()-(reference_logits_table.masked_change.round(4).std()*2)

reference_logits_table['significant_masked_change_rounded'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change.round(4)]


In [ ]:
import plotly.express as px
fig = px.line(reference_logits_table, x="position", y="masked_change",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(reference_logits_table, x="position", y="significant_masked_change_rounded",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
reference_logits_table.to_csv('./Epistasis/Revisions_Omicron_Epistasis.csv',index=False)

In [ ]:
reference_logits_table = pd.read_csv('./Epistasis/Revisions_Omicron_Epistasis.csv')

In [ ]:
max_change = reference_logits_table.masked_change.mean()+(reference_logits_table.masked_change.std())
min_change = reference_logits_table.masked_change.mean()-(reference_logits_table.masked_change.std())
reference_logits_table['single_significant_masked_change'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change]

In [ ]:
reference_logits_table.to_csv('Revisions_Omicron_Epistasis_with_single_std.csv')

In [ ]:
import plotly.express as px
fig = px.scatter(reference_logits_table, x="position", y="single_significant_masked_change",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
ba1_mutated_positions = reference_logits_table.mutation.str[1:-1].astype(int).unique()
ba1_mutated_positions

In [ ]:
one_std_identified_interactions = reference_logits_table[(reference_logits_table.single_significant_masked_change.isna() == False) & (reference_logits_table.position.isin(ba1_mutated_positions)) ]

In [ ]:
two_std_identified_interactions = reference_logits_table[(reference_logits_table.significant_masked_change.isna() == False) & (reference_logits_table.position.isin(ba1_mutated_positions)) ]

In [ ]:
px.scatter(two_std_identified_interactions,x=two_std_identified_interactions.position.astype(str),y=two_std_identified_interactions.mutation.astype(str))

In [ ]:
px.scatter(one_std_identified_interactions,x=one_std_identified_interactions.position.astype(str),y=one_std_identified_interactions.mutation.astype(str))

In [ ]:
identified_interactions = one_std_identified_interactions[['position','mutation']]
identified_interactions['mutated_position'] = identified_interactions.mutation.str[1:-1].astype(int)
swap_interactions = identified_interactions[identified_interactions.position < identified_interactions.mutated_position]
swap_interactions.rename(columns={'position':'position_1','mutated_position':'position_2'},inplace=True)
swap_interactions_2 = identified_interactions[identified_interactions.position >= identified_interactions.mutated_position]
swap_interactions_2.rename(columns={'mutated_position':'position_1','position':'position_2'},inplace=True)

swap_interactions = pd.concat([swap_interactions,swap_interactions_2])
swap_interactions.to_csv('Omicron_One_Std_Interactions.csv',index=False)

In [ ]:
identified_interactions = two_std_identified_interactions[['position','mutation']]
identified_interactions['mutated_position'] = identified_interactions.mutation.str[1:-1].astype(int)
swap_interactions = identified_interactions[identified_interactions.position < identified_interactions.mutated_position]
swap_interactions.rename(columns={'position':'position_1','mutated_position':'position_2'},inplace=True)
swap_interactions_2 = identified_interactions[identified_interactions.position >= identified_interactions.mutated_position]
swap_interactions_2.rename(columns={'mutated_position':'position_1','position':'position_2'},inplace=True)

swap_interactions = pd.concat([swap_interactions,swap_interactions_2])
swap_interactions.to_csv('Omicron_Two_Std_Interactions.csv',index=False)